# Introduction

This notebook trains the baseline method for the structure identification task. It uses a spherical harmonic-based featurization of local particle environments available in [pythia](https://github.com/glotzerlab/pythia).

In [1]:
from flowws_keras_experimental.InitializeTF import InitializeTF
InitializeTF().run(None, None)

/home/paper_authors/env/default_20201216/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


1 Physical GPUs, 1 Logical GPUs


In [2]:
import collections
import itertools

import flowws
from flowws import Argument as Arg
import freud
import numpy as np
import pyriodic
import pythia

class LocalEnvironmentSphericalHarmonics(flowws.Stage):
    ARGS = [
        Arg('num_neighbors', '-n', int, 12,
           help='Number of nearest neighbors to use'),
        Arg('structures', '-s', [str],
           help='Name of structures to take'),
        Arg('size', None, int, 512,
           help='Number of particles to replicate structures up to'),
        Arg('noise', None, [float], [1e-2, 5e-2, .1],
           help='Noise standard deviation to apply to structures'),
        Arg('test_fraction', '-t', float, 0,
           help='Fraction of data to hold back as test data'),
        Arg('seed', None, int, 13,
           help='Random seed to use for shuffling training data'),
        Arg('sph_neighbor_max', None, int, 12,
           help='Maximum number of neighbors for local environments of spherical harmonics'),
        Arg('lmax', '-l', int, 12,
           help='Maximum spherical harmonic degree l to consider'),
    ]

    def run(self, scope, storage):
        np.random.seed(self.arguments['seed'])

        xs = []
        ys = []

        name_map = collections.defaultdict(lambda: len(name_map))

        structures = list(self.arguments['structures'])
        max_types = 0
        for name in structures:
            for (structure,) in pyriodic.db.query('select structure from unit_cells where name = ?', (name,)):
                pass
            max_types = max(max_types, len(set(structure.types)))

        for name in structures:
            for (structure,) in pyriodic.db.query('select structure from unit_cells where name = ?', (name,)):
                pass

            if name in name_map:
                continue

            for noise in self.arguments['noise']:
                structure = structure.rescale_shortest_distance(1)
                structure = structure.replicate_upto(self.arguments['size'])
                structure = structure.add_gaussian_noise(noise)

                q = freud.locality.AABBQuery(structure.box, structure.positions)
                qr = q.query(
                    structure.positions, dict(
                        num_neighbors=self.arguments['num_neighbors'], exclude_ii=True))
                nl = qr.toNeighborList()
                index_i = nl.query_point_indices
                index_j = nl.point_indices

                type_filtered_descriptors = []

                for same_t in [True, False]:
                    nl_filt = nl.copy()
                    if same_t:
                        nl_filt.filter(structure.types[index_i] == structure.types[index_j])
                    else:
                        nl_filt.filter(structure.types[index_i] != structure.types[index_j])

                    if len(nl_filt):
                        type_filtered_descriptors.append(pythia.spherical_harmonics.abs_neighbor_average(
                            structure.box, structure.positions,
                            neigh_max=self.arguments['sph_neighbor_max'], lmax=self.arguments['lmax'],
                            nlist=nl_filt))
                    else:
                        type_filtered_descriptors.append(None)

                placeholder = [v for v in type_filtered_descriptors if v is not None][0]*0
                for i in range(len(type_filtered_descriptors)):
                    if type_filtered_descriptors[i] is None:
                        type_filtered_descriptors[i] = placeholder
                descriptors = np.concatenate(type_filtered_descriptors, axis=-1)

                shuf = np.arange(len(descriptors))
                np.random.shuffle(shuf)
                shuf = shuf[:self.arguments['size']]

                xs.append(descriptors[shuf])
                ys.append(name_map[name])

        ys = np.repeat(ys, [len(v) for v in xs])
        xs = np.concatenate(xs, axis=0)

        shuf = np.arange(len(xs))
        np.random.shuffle(shuf)
        N_test = int(self.arguments['test_fraction']*len(shuf))
        test_split, train_split = shuf[:N_test], shuf[N_test:]
        xs_test = xs[test_split]
        ys_test = ys[test_split]
        xs = xs[train_split]
        ys = ys[train_split]

        scope['x_train'] = xs
        scope['y_train'] = ys
        scope['x_test'] = xs_test
        scope['y_test'] = ys_test
        scope['num_classes'] = len(name_map)
        scope['type_map'] = dict(name_map)
        scope['neighborhood_size'] = self.arguments['num_neighbors']


In [3]:
import flowws
from flowws_keras_experimental import *

accs = []
for seed in range(125, 130):
    w = flowws.Workflow(
        [
            LocalEnvironmentSphericalHarmonics(
                noise=[1e-3, 5e-2, 0.1],
                structures=[
                    "hP2-Mg",
                    "cI2-W",
                    "cF4-Cu",
                    "cF8-C",
                    "cF8-SZn",
                    "cP46-Si",
                    "cF136-Si",
                    "cP2-ClCs",
                ],
                size=2048,
                num_neighbors=12,
                sph_neighbor_max=12,
                test_fraction=0.2,
                seed=seed,
            ),
            MLP(hidden_widths=[64, 64], activation="relu", batch_norm=True, dropout=0.5),
            Classifier(),
            Train(epochs=128, validation_split=0.25, reduce_lr=8, early_stopping=20),
        ]
    )

    scope = w.run()

    prediction = scope['model'].predict(scope['x_test'], batch_size=128, verbose=1)
    prediction = np.argmax(prediction, axis=-1)

    acc = np.mean(prediction == scope['y_test'])

    accs.append(acc)

2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors



Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

77/77 [==============================] - 0s 1ms/step


2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors



Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00029: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

77/77 [==============================] - 0s 1ms/step


2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors



Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00028: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

77/77 [==============================] - 0s 1ms/step


2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors



Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00029: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

77/77 [==============================] - 0s 1ms/step


2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2352 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors
2200 particles have too few neighbors



Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

77/77 [==============================] - 0s 1ms/step


In [4]:
print(np.mean(accs), np.std(accs)/np.sqrt(len(accs)))

0.9614649033570701 0.0010415818844045491
